# 0. Libraries

In [ ]:
!pip install datasets -q
!pip install transformers -q
from datasets import load_dataset
import torch
from transformers import BertTokenizer, BertModel, EncoderDecoderModel, AdamW
import pandas as pd
from functools import partial
import random

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.2 MB/s eta 0:00:00


# 1. Read Data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DeClare/joint_true_claims.csv')\
  .drop('Unnamed: 0', axis=1)
df.head()

,claim,text
0,Nikolaj Coster-Waldau worked with the Fox Broa...,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...
1,Nikolaj Coster-Waldau worked with Peter Dinklage.,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...
2,Nikolaj Coster-Waldau was in a film.,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...
3,Game of Thrones (season 1) featured Danish act...,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...
4,Nikolaj Coster-Waldau played Frank Pike in Vir...,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...


In [ ]:
df.shape

(94846, 2)

# 2. Preprocess the data

In [ ]:
# Define the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
print(tokenizer.cls_token_id)
print(tokenizer.sep_token_id)
print(tokenizer.mask_token_id)
print(tokenizer.pad_token_id)

101
102
103
0


In [ ]:
df['claim_tokens'] = df.claim.apply(partial(tokenizer.encode,
                                            add_special_tokens=False))
df.head()

,claim,text,claim_tokens,masked_claim_tokens,text_tokens,input_tokens
0,Nikolaj Coster-Waldau worked with the Fox Broa...,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...,"[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[101, 24794, 3501, 3465, 2121, 1011, 24547, 28..."
1,Nikolaj Coster-Waldau worked with Peter Dinklage.,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...,"[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[24794, 103, 3465, 103, 1011, 24547, 2850, 103...","[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[101, 24794, 103, 3465, 103, 1011, 24547, 2850..."
2,Nikolaj Coster-Waldau was in a film.,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...,"[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[24794, 3501, 3465, 2121, 103, 24547, 2850, 22...","[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[101, 24794, 3501, 3465, 2121, 103, 24547, 285..."
3,Game of Thrones (season 1) featured Danish act...,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...,"[2208, 1997, 6106, 2015, 1006, 2161, 1015, 100...","[2208, 1997, 103, 2015, 1006, 2161, 1015, 1007...","[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[101, 2208, 1997, 103, 2015, 1006, 2161, 1015,..."
4,Nikolaj Coster-Waldau played Frank Pike in Vir...,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...,"[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[103, 3501, 103, 2121, 103, 103, 2850, 103, 10...","[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[101, 103, 3501, 103, 2121, 103, 103, 2850, 10..."


In [ ]:
def random_mask(tokens, tokenizer, p=0.5):
  return [t if random.random()<p else tokenizer.mask_token_id
          for t in tokens]
random_mask([1,1,1,1,1], tokenizer, p=0.5)

[1, 1, 103, 1, 1]

In [ ]:
df['masked_claim_tokens'] = df.claim_tokens.apply(
    partial(random_mask, tokenizer=tokenizer))
df.head()

,claim,text,claim_tokens,masked_claim_tokens,text_tokens,input_tokens
0,Nikolaj Coster-Waldau worked with the Fox Broa...,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...,"[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[24794, 3501, 3465, 103, 1011, 24547, 103, 103...","[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[101, 24794, 3501, 3465, 2121, 1011, 24547, 28..."
1,Nikolaj Coster-Waldau worked with Peter Dinklage.,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...,"[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[24794, 103, 3465, 103, 1011, 24547, 2850, 222...","[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[101, 24794, 103, 3465, 103, 1011, 24547, 2850..."
2,Nikolaj Coster-Waldau was in a film.,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...,"[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[24794, 103, 103, 2121, 1011, 24547, 103, 2226...","[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[101, 24794, 3501, 3465, 2121, 103, 24547, 285..."
3,Game of Thrones (season 1) featured Danish act...,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...,"[2208, 1997, 6106, 2015, 1006, 2161, 1015, 100...","[103, 1997, 6106, 103, 1006, 2161, 103, 103, 1...","[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[101, 2208, 1997, 103, 2015, 1006, 2161, 1015,..."
4,Nikolaj Coster-Waldau played Frank Pike in Vir...,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...,"[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[24794, 3501, 3465, 103, 103, 103, 2850, 103, ...","[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[101, 103, 3501, 103, 2121, 103, 103, 2850, 10..."


In [ ]:
df['text_tokens'] = df.text.apply(partial(tokenizer.encode,
                                            add_special_tokens=False,
                                            max_length=1024,
                                            truncation=True))
df.head()

In [ ]:
def append_tokens(row, tokenizer):
  return [tokenizer.cls_token_id] + row['masked_claim_tokens']\
   + [tokenizer.sep_token_id] + row['text_tokens']
# append_tokens(df.iloc[0], tokenizer)

In [ ]:
df['input_tokens'] = df.apply(partial(append_tokens, tokenizer=tokenizer), axis=1)
df.head()

In [ ]:
df.to_csv('/content/drive/MyDrive/DeClare/tokenized_input.csv', index=False)

# 3. Fine tune the BERT

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DeClare/tokenized_input.csv')
df.head()

,claim,text,claim_tokens,masked_claim_tokens,text_tokens,input_tokens
0,Nikolaj Coster-Waldau worked with the Fox Broa...,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...,"[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[101, 24794, 3501, 3465, 2121, 1011, 24547, 28..."
1,Nikolaj Coster-Waldau worked with Peter Dinklage.,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...,"[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[24794, 103, 3465, 103, 1011, 24547, 2850, 103...","[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[101, 24794, 103, 3465, 103, 1011, 24547, 2850..."
2,Nikolaj Coster-Waldau was in a film.,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...,"[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[24794, 3501, 3465, 2121, 103, 24547, 2850, 22...","[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[101, 24794, 3501, 3465, 2121, 103, 24547, 285..."
3,Game of Thrones (season 1) featured Danish act...,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...,"[2208, 1997, 6106, 2015, 1006, 2161, 1015, 100...","[2208, 1997, 103, 2015, 1006, 2161, 1015, 1007...","[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[101, 2208, 1997, 103, 2015, 1006, 2161, 1015,..."
4,Nikolaj Coster-Waldau played Frank Pike in Vir...,Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰ...,"[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[103, 3501, 103, 2121, 103, 103, 2850, 103, 10...","[24794, 3501, 3465, 2121, 1011, 24547, 2850, 2...","[101, 103, 3501, 103, 2121, 103, 103, 2850, 10..."


In [ ]:
df['input_tokens']=df['input_tokens'].apply(eval)
df['claim_tokens']=df['claim_tokens'].apply(eval)

In [ ]:
model_name = 'bert-base-uncased'
model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 3.1 Sample embedding

In [ ]:
sample_input=df.input_tokens[0]

In [ ]:
sample_tensor = torch.tensor([sample_input])

In [ ]:
with torch.no_grad():
    outputs = model(sample_tensor)

In [ ]:
outputs[0]

tensor([[[-0.3477,  0.0386, -0.1208,  ...,  0.4032,  0.3614, -0.0886],
         [-0.6066,  0.7120,  0.6437,  ..., -0.2065,  0.0663, -0.1210],
         [-0.1051,  0.6998,  0.9174,  ...,  0.0413,  0.0195, -0.2994],
         ...,
         [ 0.6307, -0.1869,  1.0453,  ..., -0.1943,  0.0359, -0.4578],
         [ 0.1102, -0.9498,  0.7889,  ..., -0.2327, -0.1468,  0.3403],
         [-0.3126, -0.2248,  0.1638,  ...,  0.4195,  0.2126, -0.7708]]])

In [ ]:
outputs[0].shape

torch.Size([1, 295, 768])

## 3.2 Encode the entire input and target

In [ ]:
input_tensor=torch.tensor(df.input_tokens.to_numpy())